<a href="https://colab.research.google.com/github/CarolineAndradeR/Data-Science/blob/main/S%C3%A9ries_Temporais.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Conceitos**

Na área de estudos de séries temporais observar o passado é funamental para verificação de tendências e projeções de metas.

Cinco aplicações para séries temporais:
*   Projeção de valores futuros de uma série
*   Definição da sequência de entradas e saídas de sistemas dinâmicos.
*   Realizar análise de interveções, como políticas públicas ou política de preços.
*   Verifiar a correlação dinâmica entre diversas variáveis no tempo.
*   Analisar controles de sistemas e potencial de desvios.

Uma série temporal é uma sequeência de observações ordenadas no tempo.

Graficamente, uma série temporal é representada com o eixo x, o tempo e o eixo y, a medida observada. Considerabdo o contexto de modelo de regressão linear, bastaríamos ajustar uma reta em que a variável dependente seria a medida observada e a variável independente, o tempo.

O modelo clássico de regressão é insuficiente para explicar toda a dinâmica existente em uma série temporal já que não captura estrutura relacionadas com o tempo. Uma dessas dinâmicas é o fenômeno da possível correlação existente entre os dias passado, possibilitando estimar um dos modelos mais conhecidos o ARIMA.

**ARIMA**
Se tornou bastante utilizado devido á sua simplicidade de estimação, a captura dessas três estruturas possibilta um melhor ajuste da série e, consequentemente, a capacidade de previsão.

AR - Autoregressive
I - Intefrated
MA - Moving Average

Para ajustar um modelo ARIMA são necessários seis passos:
* Plotar os dados.
* Verificar a necessidade de transforma os dados (verificar estacionariedade).
* Identficar as ordens de dependência.
* Estimar os parâmetros.
* Diagnostcar a qualidade do modelo.


